1. Depth of a well balanced binary tree with m leaves is equal to

`log₂(_m_) = log(m)/log(2)`

Therefore, 1M instances => Depth = log(10^6) = `>=20`

2. A node's Gini impurity is mostly lower than it's parent. Since the `CART` training algorithm splits the node in such a way that the weighted sum of it's children's impurity is less than parent's. But, there can be some cases where a node can have higher impurity than parent but it should be compensated by the other child node being pure or very less, such that the weighted sum will be less than the parent node.

3. Decreasing `max_depth` => constraining the model, i.e regularizing it.

4. Decision Trees do not care if the data is scaled or unscaled i.e one of the best things about them. Scaling the data if the decision tree is underfitting is a waste of time.

5. Computational Complexity is _O_(_n_ × _m_ log₂(_m_)). o if you multiply the training set size by 10, the training time will be multiplied by _K_ = (_n_ × 10 _m_ × log₂(10 _m_)) / (_n_ × _m_ × log₂(_m_)) = 10 × log₂(10 _m_) / log₂(_m_). If _m_ = 10<sup>6</sup>, then _K_ ≈ 11.7, so you can expect the training time to be roughly 11.7 hours.

6. If the number of features will double, the time will also double roughly.




## Training and Fine - tuning a Decision Tree

In [1]:
# Import the data
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

# Split the data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
# Using GridSearchCV to find the best parameters
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


parameters = {
    'max_depth': list(range(2, 10)),
    'max_leaf_nodes': list(range(2, 50)),
    'min_samples_leaf': list(range(2, 6))}

dt = DecisionTreeClassifier()

gsc = GridSearchCV(dt, parameters)

gsc.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_leaf': [2, 3, 4, 5]})

In [3]:
gsc.best_estimator_

DecisionTreeClassifier(max_depth=7, max_leaf_nodes=23, min_samples_leaf=4)

In [17]:
from sklearn.metrics import accuracy_score

y_pred = gsc.predict(X_test)
accuracy_score(y_test, y_pred)

0.8735

# Creating a Random Forest

In [12]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)

for mini_train_index, mini_test_index in rs.split(X_train):
  X_mini_train = X_train[mini_train_index]
  y_mini_train = y_train[mini_train_index]
  mini_sets.append((X_mini_train, y_mini_train))


In [13]:
from sklearn.base import clone
import numpy as np

forest = [clone(gsc.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
  tree.fit(X_mini_train, y_mini_train)

  y_pred = tree.predict(X_test)
  accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.807038

In [14]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
  Y_pred[tree_index] = tree.predict(X_test)

In [15]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [16]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.8695